In [1]:
#analyze the mitochondrial translation genes between S.Cer and K.Lac to validate the results from Regev's study

In [1]:
import os
location_input = input("what computer are you on? a = Bens, b = gpucluster, c = other   ")
location_dict = {'a': "C:\\Users\Ben\Documents\GitHub\expression_broad_data", 'b': "/home/heineike/github/expression_broad_data",'c':'you need to add your location to the location_dict'}
base_dir = location_dict[location_input]
print("base directory is " + base_dir)
os.chdir(base_dir) 

what computer are you on? a = Bens, b = gpucluster, c = other   b
base directory is /home/heineike/github/expression_broad_data


In [245]:
from core import expression_plots 
from core import io_library 
# from IPython.core.debugger import Tracer
import numpy as np
import pandas as pd
import subprocess
import plotly.plotly as py
py.sign_in('heineike02', 'APPjKrtARaN2ZgUYIkqr')
from collections import Counter
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
# Learn about API authentication here: https://plot.ly/pandas/getting-started
# Find your api_key here: https://plot.ly/settings/api


In [3]:
%load_ext autoreload
%autoreload 2
#%matplotlib inline
%matplotlib notebook

In [4]:
#Import Mitochondrial genes
mito_fname =  base_dir+os.sep + os.path.join("expression_data","go_terms","mitochondrial_translation_annotations.txt")
mito_table = pd.read_table(mito_fname,header = 7)
mito_genes = list(set([gene for gene in mito_table['Gene Systematic Name'] if gene[0]=='Y']))

In [5]:
#Visualize expression differences between species on my dataset
#load kl_sc_PKA_data
kl_sc_PKA_data = pd.read_pickle(base_dir + os.sep + os.path.join("expression_data","kl_PKA_as_20160824","kl_sc_PKA_data"))


In [6]:

kl_sc_PKA_data_mito = kl_sc_PKA_data[kl_sc_PKA_data['sc_genename'].isin(mito_genes)]
kl_sc_PKA_data_notmito = kl_sc_PKA_data[~kl_sc_PKA_data['sc_genename'].isin(mito_genes)]

fig = {
    'data': [
  		{
        	'x': kl_sc_PKA_data_notmito['KL_PKA(AS)+1NMPP1'], 
        	'y': kl_sc_PKA_data_notmito['SC_PKA(AS)+1NMPP1'], 
        	'text': kl_sc_PKA_data_notmito['SC_common_name'], 
        	'mode': 'markers', 
        	'name': 'other genes'},
  		{
        	'x': kl_sc_PKA_data_mito['KL_PKA(AS)+1NMPP1'], 
        	'y': kl_sc_PKA_data_mito['SC_PKA(AS)+1NMPP1'], 
        	'text': kl_sc_PKA_data_mito['SC_common_name'], 
        	'mode': 'markers', 
        	'name': 'mitochondrial translation'}
            ],
    'layout': {
        'xaxis': {'title': 'KL LFC 4uM NMPP1'},
        'yaxis': {'title': "SC averaged Fold Change"}
    }
}

url = py.plot(fig, filename='Mitochondrial translation genes')



In [11]:

kl_sc_PKA_data.columns


Index(['N SCer Orthologs', 'kl_genename', 'sc_genename', 'KL_PKA(AS)+1NMPP1',
       'padj', 'SC_PKA(AS)+1NMPP1', 'SC_common_name'],
      dtype='object')

In [7]:
#Import promoters for mito genes
#visualize SNF1 sites

#Read in the KL promoter database.  
kl_promoters = pd.read_pickle(base_dir + os.sep + os.path.join("expression_data","kl_promoters","kl_promoters.pkl"))

#Read in the SC promoter database.  
sc_promoters = pd.read_pickle(base_dir + os.sep + os.path.join("expression_data","sc_promoters","sc_promoters.pkl"))

# Make a subset of kl promoters for mitochondrial translation.  
mito_genes_kl_orth = list(set(kl_sc_PKA_data[kl_sc_PKA_data['sc_genename'].isin(mito_genes)]['kl_genename']))
#including set here removes duplicates

kl_promoters_mito = kl_promoters.loc[mito_genes_kl_orth,]

fname = base_dir + os.sep + os.path.join("expression_data", "regev_analysis", "mito_promoters_kl.fasta")
prom_seq_column = 4
with open(fname,'w') as f: 
    for row in kl_promoters_mito.itertuples():
        header_line = '>' + row[0] + ' 700bp_upstream\n'
        seq_line = row[prom_seq_column] + '\n'
        f.write(header_line)
        f.write(seq_line)

# Make a subset of sc promoters for mitochondrial translation.  
sc_promoters_mito = sc_promoters.loc[mito_genes,]

fname = base_dir + os.sep + os.path.join("expression_data", "regev_analysis", "mito_promoters_sc.fasta")
prom_seq_column = 2
with open(fname,'w') as f: 
    for row in sc_promoters_mito.itertuples():
        header_line = '>' + row[0] + ' 700bp_upstream\n'
        seq_line = row[prom_seq_column] + '\n'
        f.write(header_line)
        f.write(seq_line)

In [10]:
#Use subprocess to run meme commands: 

#ame --verbose 1 --oc . --control all_kl_promoters.fasta --bgformat 1 --scoring avg --method ranksum --pvalue-report-threshold 0.05 mito_promoters_kl.fasta db/JASPAR/JASPAR_CORE_2016_fungi.meme
motif_db = "/home/kieran/memeTest/JASPAR_CORE_2016_fungi.meme"
input_sequences = base_dir + "/expression_data/regev_analysis/mito_promoters_kl.fasta"
control_sequences = base_dir + "/expression_data/kl_promoters/all_kl_promoters.fasta"
output_dir = base_dir + "/expression_data/regev_analysis/ame_out"
file_prefix = "kl_mito_"

ame_command = [ "/home/kieran/meme/bin/ame",
              "--verbose", "2",
              "--oc", output_dir,
              "--control", control_sequences,
              "--bgformat", "1", 
              "--scoring", "totalhits",
              "--method", "fisher", 
              "--pvalue-report-threshold", "0.05", 
              input_sequences,
              motif_db]

ame_output = subprocess.run(ame_command,stdout = subprocess.PIPE) 

print(ame_output.returncode)

#change file prefix
for fname in ["ame.txt","ame.html"]:
    os.rename(output_dir + os.sep + fname, output_dir + os.sep + file_prefix + fname)


0


In [11]:
#Use subprocess to run meme commands: 

motif_db = "/home/kieran/memeTest/JASPAR_CORE_2016_fungi.meme"
input_sequences = base_dir + "/expression_data/regev_analysis/mito_promoters_sc.fasta"
control_sequences = base_dir + "/expression_data/sc_promoters/all_sc_promoters.fasta"
output_dir = base_dir + "/expression_data/regev_analysis/ame_out"
file_prefix = "sc_mito_"

ame_command = [ "/home/kieran/meme/bin/ame",
              "--verbose", "2",
              "--oc", output_dir,
              "--control", control_sequences,
              "--bgformat", "1", 
              "--scoring", "totalhits",
              "--method", "fisher", 
              "--pvalue-report-threshold", "0.05", 
              input_sequences,
              motif_db]

ame_output = subprocess.run(ame_command,stdout = subprocess.PIPE) 

print(ame_output.returncode)

#change file prefix
for fname in ["ame.txt","ame.html"]:
    os.rename(output_dir + os.sep + fname, output_dir + os.sep + file_prefix + fname)

0


In [65]:
#Run fimo on the promoters for the mitochondrial translation genes for sum1, sfp1, stb3, abf1

#fimo --oc . --verbosity 1 --thresh 1.0E-4 abf1_stb3_sum1_sfp1.meme mito_promoters_kl.fasta



#Run fimo on all the kl promoters for the mitochondrial translation genes

#Make a dataframe 








0